In [ ]:
import glob
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import json
import glob
import matplotlib.pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES']='2'

DP_DIR = './shuffle_data_gzip/'



In [ ]:
BASE_SIZE = 256
NCSVS = 100
NCATS = 340
np.random.seed(seed=1987)



STEPS = 800
EPOCHS = 16
size = 64
batchsize = 680
NCATS =340

In [3]:
fileList = glob.glob("./shuffle_data_gzip/*.csv.gz")     

In [4]:
fileList[1]

'./shuffle_data_gzip/train_k52.csv.gz'

In [5]:
# pd.read_csv('./shuffle_data_gzip/train_k52.csv.gz',sep=',').shape

In [6]:
def parse_csv(line):
    column_default = [tf.constant("0",dtype=tf.string), #drawing
                      tf.constant(0,dtype=tf.int32)] #label
    columns = tf.io.decode_csv(line, column_default, select_cols=[1,5])
    features = columns[0]
    label = columns[1]
    return features, label



In [7]:
def draw_cv2(raw_strokes, size=64, lw=6):
    raw_strokes = eval(raw_strokes.numpy())
    img = np.zeros((256, 256), np.uint8)
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    return cv2.resize(img, (size, size)) 

In [8]:
def tf_draw_cv2(image, label):
    [image] = tf.py_function(draw_cv2, [image], [tf.float32])
    image = tf.reshape(image,(64,64,1))
    label = tf.one_hot(label,depth=NCATS)
    image.set_shape((64,64,1))
    label.set_shape((340,))
    return image, label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(fileList[:-1])
train_ds = train_ds.interleave(lambda x:
    tf.data.TextLineDataset(x,compression_type='GZIP').skip(1).map(parse_csv,num_parallel_calls=tf.data.experimental.AUTOTUNE),
    cycle_length=4, block_length=16,num_parallel_calls=tf.data.experimental.AUTOTUNE)

#drawing处理
train_ds = train_ds.map(tf_draw_cv2,num_parallel_calls=tf.data.experimental.AUTOTUNE)
# tensor 



train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE).shuffle(3000).batch(1024)

In [ ]:
# train_ds = tf.data.TextLineDataset(fileList[5],compression_type='GZIP').skip(1).map(parse_csv)  
# train_ds = train_ds.map(tf_draw_cv2)
# train_ds = train_ds.shuffle(3000).batch(1024)



In [ ]:
# import time
# import tqdm
# for file in fileList:
#     try:
#         train_ds = tf.data.TextLineDataset(file,compression_type='GZIP').skip(1).map(parse_csv)  
#         train_ds = train_ds.map(tf_draw_cv2,num_parallel_calls=tf.data.experimental.AUTOTUNE)
#         train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE).shuffle(3000).batch(10240)

#         for image,label in tqdm.tqdm(train_ds):
#             time.sleep(0.0000000000001)
#     except:
#         print(file)

338it [10:24,  1.77s/it]

In [6]:
# def parse_csv(line):
#     column_default = [tf.constant("0",dtype=tf.string),
#                       tf.constant("0",dtype=tf.string),
#                       tf.constant("0",dtype=tf.string),
#                       tf.constant("0",dtype=tf.string),
#                       tf.constant("0",dtype=tf.string),
#                       tf.constant(0,dtype=tf.int32),
#                       tf.constant(0,dtype=tf.int32)]
#     columns = tf.io.decode_csv(line, column_default)
#     label = columns[-2] 
#     features = columns[1]
#     return features, label



In [7]:
# def draw_cv2(raw_strokes, size=64, lw=6):
#     raw_strokes = eval(raw_strokes.numpy())
#     img = np.zeros((256, 256), np.uint8)
#     for stroke in raw_strokes:
#         for i in range(len(stroke[0]) - 1):
#             _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
#     return cv2.resize(img, (size, size)) 

In [8]:
# def tf_draw_cv2(image, label):
#     [image] = tf.py_function(draw_cv2, [image], [tf.float32])
#     image = tf.reshape(image,(64,64,1))
#     label = tf.one_hot(label,depth=NCATS)
#     image.set_shape((64,64,1))
#     label.set_shape((340,))
#     return image, label

In [8]:
# train_ds = tf.data.TextLineDataset(fileList[5],compression_type='GZIP').skip(1).map(parse_csv)  
# train_ds = train_ds.map(tf_draw_cv2)
# train_ds = train_ds.shuffle(3000).batch(10240)

In [ ]:
# import time
# import tqdm
# for file in fileList:
#     try:
#         train_ds = tf.data.TextLineDataset(file,compression_type='GZIP').skip(1).map(parse_csv)  
#         train_ds = train_ds.map(tf_draw_cv2)
#         train_ds = train_ds.shuffle(3000).batch(10240)

#         for image,label in tqdm.tqdm(train_ds):
#             time.sleep(0.0000000000001)
#     except:
#         print(file)

MobileNet论文参考： https://arxiv.org/pdf/1704.04861.pdf

tf.keras.applications.mobilenet.MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
- input_shape: 可选，输入尺寸元组，仅当 include_top=False 时有效，否则输入形状必须是 (224, 224, 3)（channels_last 格式）或 (3, 224, 224)（channels_first 格式）。它必须为 3 个输入通道，且宽高必须不小于 32，比如 (200, 200, 3) 是一个合法的输入尺寸。
- alpha: 控制网络的宽度：
- 如果 alpha < 1.0，则同比例减少每层的滤波器个数。
- 如果 alpha > 1.0，则同比例增加每层的滤波器个数。
- 如果 alpha = 1，使用论文默认的滤波器个数
- depth_multiplier: depthwise卷积的深度乘子，也称为（分辨率乘子）
- dropout: dropout 概率
- include_top: 是否包括顶层的全连接层。
- weights: None 代表随机初始化， 'imagenet' 代表加载在 ImageNet 上预训练的权值。
- input_tensor: 可选，Keras tensor 作为模型的输入（比如 layers.Input() 输出的 tensor）。
- pooling: 可选，当 include_top 为 False 时，该参数指定了特征提取时的池化方式。 None 代表不池化，直接输出最后一层卷积层的输出，该输出是一个四维张量。'avg' 代表全局平均池化（GlobalAveragePooling2D），相当于在最后一层卷积层后面再加一层全局平均池化层，输出是一个二维张量。 'max' 代表全局最大池化
- classes: 可选，图片分类的类别数，仅当 include_top 为 True 并且不加载预训练权值时可用。

In [9]:

class MobileNetModel(tf.keras.models.Model):
    def __init__(self, size, n_labels, **kwargs):
        super(MobileNetModel, self).__init__(**kwargs)
        self.base_model = tf.keras.applications.MobileNet(input_shape=(size, size, 1), include_top=False, weights=None, classes=n_labels)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(1024, activation='relu')
        self.outputs =  tf.keras.layers.Dense(n_labels, activation='softmax')

        
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        output_ = self.outputs(x)
        return output_


In [10]:
model = MobileNetModel(size=64,n_labels=NCATS)

loss_object = tf.keras.losses.CategoricalCrossentropy()


learning_rate = 0.002
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=3,name='train_top_3_categorical_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=3,name='test_top_3_categorical_accuracy')

# @tf.function
#setp
def train_one_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
     
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    train_top3_accuracy(labels, predictions)
#step
def val_one_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_top3_accuracy(labels, predictions)

In [11]:
import time

In [13]:
for a,b in train_ds.take(1):
    print(a.shape,b.shape)

(1024, 64, 64, 1) (1024, 340)


In [14]:
val_ds = tf.data.TextLineDataset(fileList[-1],compression_type='GZIP').skip(1).map(parse_csv,num_parallel_calls=tf.data.experimental.AUTOTUNE)  
val_ds = val_ds.map(tf_draw_cv2,num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.batch(1024)

In [ ]:
EPOCHS=10
for epoch in range(EPOCHS):
    # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_top3_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_top3_accuracy.reset_states()

    for step,(images, labels) in enumerate(train_ds):
        train_one_step(images, labels)
        
        if step % 200 == 0:
            print("step:{0}; Samples:{1}; Train Loss:{2}; Train Accuracy:{3},Train Top3 Accuracy:{4}".format(step, (step + 1) * 1024, 
                                                                                                             train_loss.result(), 
                                                                                                             train_accuracy.result()*100, 
                                                                                                             train_top3_accuracy.result()*100))
        if step >1000:
            break




    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          train_top3_accuracy()*100,
                          test_loss.result(),
                          test_accuracy.result() * 100,
                          test_top3_accuracy()*100
                         ))

# 读取数据
- tf.data.TextLineDataset解析文件
- tf.py_function 作用
- tf.io.decode_csv
